## Qualitative Evaluation Example

In [ ]:
%matplotlib inline

import random
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import torch
import dense_correspondence_manipulation.utils.utils as utils
utils.add_dense_correspondence_to_python_path()

import dense_correspondence
from dense_correspondence.evaluation.evaluation import *
import dense_correspondence.correspondence_tools.correspondence_plotter as correspondence_plotter
from dense_correspondence.dataset.dense_correspondence_dataset_masked import ImageType

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

In [ ]:
config_filename = os.path.join(utils.getDenseCorrespondenceSourceDir(), 'config', 
                               'dense_correspondence', 'evaluation', 'evaluation.yaml')
config = utils.getDictFromYamlFilename(config_filename)
default_config = utils.get_defaults_config()

# utils.set_cuda_visible_devices([0])
dce = DenseCorrespondenceEvaluation(config)
DCE = DenseCorrespondenceEvaluation

network_name = "robot_box_3"
dcn = dce.load_network_from_config(network_name)
dataset = dcn.load_training_dataset()


In [ ]:
# This is the code of original author
#DenseCorrespondenceEvaluation.evaluate_network_qualitative(dcn, dataset=dataset, randomize=True)

In [ ]:
from dense_correspondence.correspondence_tools.correspondence_finder import random_sample_from_masked_image
def draw_compare_original(dcn, dataset, scene_name, img_a_idx, img_b_idx, num_matches=10):
    """copy from single_image_pair_qualitative_analysis"""
    rgb_a, _, mask_a, _ = dataset.get_rgbd_mask_pose(scene_name, img_a_idx)
    rgb_b, _, mask_b, _ = dataset.get_rgbd_mask_pose(scene_name, img_b_idx)

    mask_a = np.asarray(mask_a)
    mask_b = np.asarray(mask_b)

    # compute dense descriptors
    rgb_a_tensor = dataset.rgb_image_to_tensor(rgb_a)
    rgb_b_tensor = dataset.rgb_image_to_tensor(rgb_b)

    # these are Variables holding torch.FloatTensors, first grab the data, then convert to numpy
    res_a = dcn.forward_single_image_tensor(rgb_a_tensor).data.cpu().numpy()
    #load here
    res_b = dcn.forward_single_image_tensor(rgb_b_tensor).data.cpu().numpy()
    #res_b = DenseCorrespondenceEvaluation.load_pre_computed_feature_map(scene_name, img_b_idx)
    print (scene_name)


    # sample points on img_a. Compute best matches on img_b
    # note that this is in (x,y) format
    # TODO: if this mask is empty, this function will not be happy
    # de-prioritizing since this is only for qualitative evaluation plots
    sampled_idx_list = random_sample_from_masked_image(mask_a, num_matches)

    # list of cv2.KeyPoint
    kp1 = []
    kp2 = []
    matches = []  # list of cv2.DMatch

    # placeholder constants for opencv
    diam = 0.01
    dist = 0.01
    
    for i in xrange(0, num_matches):
        # convert to (u,v) format
        pixel_a = [sampled_idx_list[1][i], sampled_idx_list[0][i]]
        best_match_uv, best_match_diff, norm_diffs =\
            DenseCorrespondenceNetwork.find_best_match(pixel_a, res_a, res_b)

        # be careful, OpenCV format is  (u,v) = (right, down)
        kp1.append(cv2.KeyPoint(pixel_a[0], pixel_a[1], diam))
        kp2.append(cv2.KeyPoint(best_match_uv[0], best_match_uv[1], diam))
        matches.append(cv2.DMatch(i, i, dist))
        
    gray_a_numpy = cv2.cvtColor(np.asarray(rgb_a), cv2.COLOR_BGR2GRAY)
    gray_b_numpy = cv2.cvtColor(np.asarray(rgb_b), cv2.COLOR_BGR2GRAY)
    img3 = cv2.drawMatches(gray_a_numpy, kp1, gray_b_numpy, kp2, matches, flags=2, outImg=gray_b_numpy)
    fig, axes = plt.subplots(nrows=1, ncols=1)
    fig.set_figheight(10)
    fig.set_figwidth(15)
    axes.imshow(img3)

    return sampled_idx_list

In [ ]:
def draw_compare_flow(dcn, dataset, scene_name, img_a_idx, img_b_idx, flow_threshold, sampled_idx_list=None, num_matches=10):
    """copy from single_image_pair_qualitative_analysis"""
    rgb_a, _, mask_a, _ = dataset.get_rgbd_mask_pose(scene_name, img_a_idx)
    rgb_b, _, mask_b, _ = dataset.get_rgbd_mask_pose(scene_name, img_b_idx)

    mask_a = np.asarray(mask_a)
    mask_b = np.asarray(mask_b)

    # compute dense descriptors
    rgb_a_tensor = dataset.rgb_image_to_tensor(rgb_a)
    rgb_b_tensor = dataset.rgb_image_to_tensor(rgb_b)

    # these are Variables holding torch.FloatTensors, first grab the data, then convert to numpy
    res_a = dcn.forward_single_image_tensor(rgb_a_tensor).data.cpu().numpy()
    #load here
    #res_b = dcn.forward_single_image_tensor(rgb_b_tensor).data.cpu().numpy()
    res_b = DenseCorrespondenceEvaluation.load_pre_computed_feature_map(scene_name, img_b_idx)
    
    load_feature_map_folder = os.path.join(utils.getDenseCorrespondenceSourceDir(), 'data_volume', 'pdc',
                                'flow_errormap', 'example', 'itri_box')
    load_res_b_f, load_res_b_n = DenseCorrespondenceEvaluation.load_nf_feature(load_feature_map_folder, scene_name, img_b_idx)

    # sample points on img_a. Compute best matches on img_b
    # note that this is in (x,y) format
    # TODO: if this mask is empty, this function will not be happy
    # de-prioritizing since this is only for qualitative evaluation plots
    if not sampled_idx_list:
        sampled_idx_list = random_sample_from_masked_image(mask_a, num_matches)
    else:
        print ("use sampled_idx_list from input")
    print ("number of sampled_idx_list: %d" %len(sampled_idx_list[0]))

    # list of cv2.KeyPoint
    kp1 = []
    kp2 = []
    matches = []  # list of cv2.DMatch

    # placeholder constants for opencv
    diam = 0.01
    dist = 0.01

    number_wanted = 0
    for i in xrange(0, num_matches):
        # convert to (u,v) format
        pixel_a = [sampled_idx_list[1][i], sampled_idx_list[0][i]]
        best_match_uv, best_match_diff, norm_diffs =\
            DenseCorrespondenceNetwork.find_best_match(pixel_a, res_a, res_b)

        # compute best match
        ## between a & b
        uv_b_pred, best_match_diff, norm_diffs =\
            DenseCorrespondenceNetwork.find_best_match(pixel_a, res_a,
                                                       res_b, debug=False)
        ## between a & b-1
        uv_bf_pred, best_match_diff_a_bf, norm_diffs_a_bf =\
            DenseCorrespondenceNetwork.find_best_match(pixel_a, res_a,
                                                       load_res_b_f, debug=False)
        
        ## between a & b+1
        uv_bn_pred, best_match_diff_a_bn, norm_diffs_a_bn =\
            DenseCorrespondenceNetwork.find_best_match(pixel_a, res_a,
                                                       load_res_b_n, debug=False)

        # compute flow(uv_bf_pred) & flow(uv_b_pred)
        load_flo_folder = os.path.join(utils.getDenseCorrespondenceSourceDir(), 'data_volume', 'pdc',
                                    'flow_errormap', 'example', 'inference', 'itri_box')
        flow_uv_bf_pred = DenseCorrespondenceEvaluation.select_pixel_flow(load_flo_folder, scene_name, 'rgbf-rgb', img_b_idx, uv_bf_pred)
        flow_uv_b_pred = DenseCorrespondenceEvaluation.select_pixel_flow(load_flo_folder, scene_name, 'rgb-rgbn', img_b_idx, uv_b_pred)
        
        # compute predicted point distance transformation. (uv_b_pred - uv_bf_pred) & (uv_bn_pred - uv_b_pred)
        dist_b_bf = np.zeros(2, np.float64)
        dist_bn_b = np.zeros(2, np.float64)
        dist_b_bf[0] = uv_b_pred[0] - uv_bf_pred[0]
        dist_b_bf[1] = uv_b_pred[1] - uv_bf_pred[1]
        dist_bn_b[0] = uv_bn_pred[0] - uv_b_pred[0]
        dist_bn_b[1] = uv_bn_pred[1] - uv_b_pred[1]
        
        error_flow = 0.5*np.linalg.norm(flow_uv_bf_pred - dist_b_bf)+0.5*np.linalg.norm(flow_uv_b_pred - dist_bn_b)
        
        if error_flow < flow_threshold:
            kp1.append(cv2.KeyPoint(pixel_a[0], pixel_a[1], diam))
            kp2.append(cv2.KeyPoint(best_match_uv[0], best_match_uv[1], diam))
            matches.append(cv2.DMatch(number_wanted, number_wanted, dist))
            number_wanted += 1

    print ("number point: %d %d" % (len(kp1), len(kp2)))
    gray_a_numpy = cv2.cvtColor(np.asarray(rgb_a), cv2.COLOR_BGR2GRAY)
    gray_b_numpy = cv2.cvtColor(np.asarray(rgb_b), cv2.COLOR_BGR2GRAY)
    img3 = cv2.drawMatches(gray_a_numpy, kp1, gray_b_numpy, kp2, matches, flags=2, outImg=gray_b_numpy)
    fig, axes = plt.subplots(nrows=1, ncols=1)
    fig.set_figheight(10)
    fig.set_figwidth(15)
    axes.imshow(img3)


In [ ]:
scene_name = "2018-11-07-16-18-28"
img_a_idx = 3
img_b_idx = 21
prevous_sampled_idx_list = draw_compare_original(dcn, dataset, scene_name, img_a_idx, img_b_idx, num_matches=50)
draw_compare_flow(dcn, dataset, scene_name, img_a_idx, img_b_idx, 
                  flow_threshold = 8.43, sampled_idx_list=prevous_sampled_idx_list, num_matches=50)

In [ ]:
scene_name = "2018-11-07-17-29-42"
img_a_idx = 2
img_b_idx = 18
prevous_sampled_idx_list = draw_compare_original(dcn, dataset, scene_name, img_a_idx, img_b_idx, num_matches=100)
draw_compare_flow(dcn, dataset, scene_name, img_a_idx, img_b_idx, 
                  flow_threshold = 8.43, sampled_idx_list=prevous_sampled_idx_list, num_matches=100)